In [1]:
import pandas as pd
import numpy as np
import re
import time

import requests
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
url = 'https://ballotpedia.org/List_of_congressional_candidates_in_the_2022_elections'

In [3]:
state_names = ['Alaska', 'Alabama', 'Arkansas', 'Arizona', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Iowa', 'Idaho', 'Illinois', 'Indiana', 'Kansas', 'Kentucky', 'Louisiana', 'Massachusetts', 'Maryland', 'Maine', 'Michigan', 'Minnesota', 'Missouri', 'Mississippi', 'Montana', 'North Carolina', 'North_Carolina', 'North Dakota', 'North_Dakota', 'Nebraska', 'New Hampshire', 'New_Hampshire', 'New Jersey', 'New_Jersey', 'New Mexico', 'New_Mexico', 'Nevada', 'New York', 'New_York', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'Rhode_Island', 'South Carolina', 'South_Carolina', 'South Dakota', 'South_Dakota', 'Tennessee', 'Texas', 'Utah', 'Virginia', 'Vermont', 'Washington', 'Wisconsin', 'West Virginia', 'West_Virginia', 'Wyoming']
state_regex = '(' + ')|('.join(state_names) + ')'
party_names = ['Democratic Party','Republican Party','Libertarian Party','Independent','Nonpartisan','Unaffiliated','Alaskan Independence Party','American Independent Party','Green Party','No party preference','Peace and Freedom Party of California','American Constitution Party','Independent Party','No Party Affiliation','Constitution Party','Working Class Party','Legal Marijuana Now Party','Working Families Party','Independence Party','Progressive Party','United Utah Party','Vermont Progressive Party','American Solidarity Party','Congress Sucks Party','Logic Party','Alliance Party']
party_regex = '(' + ')|('.join(party_names) + ')'

In [4]:
## launch headless Edge instance
options = webdriver.EdgeOptions()
options.add_argument('--headless')

driver = webdriver.Edge(options = options)
driver.get(url)

## grab html from page
page = driver.page_source.encode('utf-8')
soup = BeautifulSoup(page, 'html.parser')

In [5]:
## loop through states, grab code, and grab links for Senate and House
senate_links = []
house_links = []

for state in state_names:
    state_html = soup.find('div', id = state.replace(' ', '_'))

    senate_html = str(state_html).split('id="U.S._House')[0]
    senate_links.extend(re.findall(r'(?<=href=")https://ballotpedia.org/[A-Za-z\.\'\-\(\)_]+(?=">)', 
                                   str(senate_html))
                       )
    
    house_html = str(state_html).split('id="U.S._House')[1]
    # house_links.extend(re.findall(r'(?<=")(https://ballotpedia.org/([A-Za-z\.\'\-]+_){1,4}[A-Za-z\.\'\-\(\)]+)(?=")',
    house_links.extend(re.findall(r'(?<=href=")https://ballotpedia.org/[A-Za-z\.\'\-\(\)_]+(?=">)',
                                  str(house_html))
                      )

In [24]:
house_links = [x for x in house_links if '_Party' not in x and '/Nonpartisan' not in x]
house_links = [*set(house_links)]

In [25]:
len(house_links)

2389

In [17]:
# function
def scrape_candidate_page(url, type = 'House'):
    url = url
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    body = soup.find('div', class_ = 'mw-parser-output')
    link_table = soup.find('div', class_ = 'infobox person')
    election_link = body.find('a', href = re.compile(str('https://ballotpedia.org/.*(Congress|House).*,_2022')))

    name = re.search(r'[A-Za-zé\.\'\- ]+(?=\n\n)', link_table.get_text())[0]
    try:
        party = re.search(party_regex, link_table.get_text())[0]
    except:
        party = ''
    try:
        state = re.search(state_regex, str(election_link))[0]
    except:
        state = ''
    try:
        district = re.search(r'(?<=_)[0-9]+(?=[a-z][a-z]_)', str(election_link))[0]
    except:
        district = ''
    try:
        last_election = re.search(r'[A-Za-z]+ [0-9]+, [0-9]+(?=\n\n)', link_table.get_text())[0]
    except:
        last_election = ''
    twitter = re.findall(r'(?<=")https://www.twitter.com/\w+(?=")', str(link_table))
    ballotpedia = url
    
    result = [name, party, state, district, last_election, twitter, ballotpedia]
    
    return result #, body, link_table, election_link;

In [18]:
# result, body, link_table, election_link = scrape_candidate_page(house_links[1967])
result = scrape_candidate_page(house_links[1967])

In [19]:
result

['Priscilla Taylor',
 'Democratic Party',
 'Florida',
 '20',
 'November 2, 2021',
 [],
 'https://ballotpedia.org/Priscilla_Taylor']

In [26]:
ballotpedia_scrape = pd.DataFrame(columns = ['name', 'party', 'state', 'district', 'last_election', 'twitter', 'ballotpedia'])

for url in house_links:
    result = scrape_candidate_page(url)
    ballotpedia_scrape.loc[len(ballotpedia_scrape)] = result
    # time.sleep(1)
    if house_links.index(url) % 100 == 0:
        print(f'Progress - {house_links.index(url)} of {len(house_links)}')

Progress - 0 of 2389
Progress - 100 of 2389
Progress - 200 of 2389
Progress - 300 of 2389
Progress - 400 of 2389
Progress - 500 of 2389
Progress - 600 of 2389
Progress - 700 of 2389
Progress - 800 of 2389
Progress - 900 of 2389
Progress - 1000 of 2389
Progress - 1100 of 2389
Progress - 1200 of 2389
Progress - 1300 of 2389
Progress - 1400 of 2389
Progress - 1500 of 2389
Progress - 1600 of 2389
Progress - 1700 of 2389
Progress - 1800 of 2389
Progress - 1900 of 2389
Progress - 2000 of 2389
Progress - 2100 of 2389
Progress - 2200 of 2389
Progress - 2300 of 2389


In [27]:
ballotpedia_scrape

,name,party,state,district,last_election,twitter,ballotpedia
0,Steve Hohe,Libertarian Party,Kansas,3,"November 8, 2022",[https://www.twitter.com/stevehoheks],https://ballotpedia.org/Steve_Hohe
1,Christopher Dardzinski,,Michigan,13,"November 8, 2022",[],https://ballotpedia.org/Christopher_Dardzinski
2,Long Pham,Republican Party,California,45,"June 7, 2022",[],https://ballotpedia.org/Long_Pham_(California)
3,Diana DeGette,Democratic Party,Colorado,1,"November 8, 2022","[https://www.twitter.com/RepDianaDeGette, http...",https://ballotpedia.org/Diana_DeGette
4,Richard Watkins,Democratic Party,North Carolina,4,"May 17, 2022",[https://www.twitter.com/RLWatkinsNC],https://ballotpedia.org/Richard_Watkins
...,...,...,...,...,...,...,...
2384,Eugene Howard,Democratic Party,Texas,14,"March 1, 2022",[https://www.twitter.com/VisionaryHoward],https://ballotpedia.org/Eugene_Howard
2385,Meagan Hanson,Republican Party,Georgia,6,"May 24, 2022","[https://www.twitter.com/HansonforHouse, https...",https://ballotpedia.org/Meagan_Hanson
2386,Garret Graves,Republican Party,Louisiana,6,"November 8, 2022","[https://www.twitter.com/RepGarretGraves, http...",https://ballotpedia.org/Garret_Graves
2387,David McKinley,Republican Party,West Virginia,2,"May 10, 2022",[https://www.twitter.com/McKinleyforWV],https://ballotpedia.org/David_McKinley


In [28]:
len(ballotpedia_scrape.query('twitter.str.len() == 0'))

689

In [30]:
ballotpedia_scrape.party.value_counts()

Republican Party                         1227
Democratic Party                          850
Independent                               117
Libertarian Party                          83
                                           32
Nonpartisan                                19
No Party Affiliation                       17
Working Class Party                         8
Green Party                                 7
Constitution Party                          6
Legal Marijuana Now Party                   4
American Independent Party                  2
Working Families Party                      2
American Constitution Party                 2
United Utah Party                           2
Progressive Party                           2
Independence Party                          1
American Solidarity Party                   1
Vermont Progressive Party                   1
Unaffiliated                                1
Congress Sucks Party                        1
Alliance Party                    

In [31]:
ballotpedia_scrape.query('party == ""')

,name,party,state,district,last_election,twitter,ballotpedia
1,Christopher Dardzinski,,Michigan,13,"November 8, 2022",[],https://ballotpedia.org/Christopher_Dardzinski
28,Carl Paladino,,New York,23,"August 23, 2022",[https://www.twitter.com/CarlPaladinoNY],https://ballotpedia.org/Carl_P._Paladino
67,Allen Cannon,,New Jersey,1,"November 8, 2022",[],https://ballotpedia.org/Allen_Cannon
72,Christopher Rodriguez,,California,49,"November 6, 2018",[https://www.twitter.com/Vote4CRod],https://ballotpedia.org/Christopher_Rodriguez
94,Patricia Kline,,New Jersey,1,"November 8, 2022",[],https://ballotpedia.org/Patricia_Kline
97,Pam Daniels,,New Jersey,4,"November 8, 2022",[https://www.twitter.com/PamRDaniels],https://ballotpedia.org/Pam_Daniels
244,Pablo Olivera,,New Jersey,8,"November 8, 2022",[https://www.twitter.com/Olivera4Congres],https://ballotpedia.org/Pablo_Olivera
454,Cynthia Dianne Steel,,Nevada,1,"June 14, 2022",[],https://ballotpedia.org/Cynthia_Dianne_Steel
501,Steve Yurash,,Colorado,2,"November 8, 2022",[],https://ballotpedia.org/Steve_Yurash
534,Haroun McClellan,,Minnesota,1,"August 9, 2022",[https://www.twitter.com/HarounMcclellan],https://ballotpedia.org/Haroun_McClellan


In [32]:
ballotpedia_scrape.query('state == ""')

,name,party,state,district,last_election,twitter,ballotpedia


In [33]:
# Alaska, Delaware, North Dakota, South Dakota, Vermont, Wyoming
ballotpedia_scrape.query('district == ""').state.unique()

array(['Alaska', 'North_Dakota', 'Delaware', 'Wyoming', 'Vermont',
       'South_Dakota'], dtype=object)

In [34]:
ballotpedia_scrape.district.value_counts()

1     244
2     238
3     182
4     181
6     147
5     142
7     132
8     110
10     97
       85
9      83
13     64
11     64
12     51
14     44
15     41
17     31
20     29
23     28
16     28
30     27
22     25
21     21
35     20
27     19
28     19
24     17
26     17
34     17
37     17
38     16
18     16
25     14
19     14
32     14
36     10
33      9
39      7
49      7
29      6
46      6
31      6
42      6
50      5
41      5
47      5
51      4
48      4
40      4
43      4
45      3
44      2
52      2
Name: district, dtype: int64

In [36]:
ballotpedia_scrape.to_csv('data/ballotpedia_scrape2022.csv', index = False)

In [ ]:
# url = 'https://ballotpedia.org/List_of_congressional_candidates_in_the_2022_elections'
# page = requests.get(url)
# soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
# html_text = soup.prettify()
# with open('output/ballotpedia_html.txt', 'w', encoding = 'utf-8') as text_file:
#     print(html_text, file = text_file)

In [ ]:
# candidate_links = re.findall(r'(?<=")(https://ballotpedia.org/([A-Za-z\.\']+_){1,4}[A-Za-z\.\'\(\)]+)(?=")', str(soup))
# candidate_links = [x[0] for x in candidate_links if '_Party' not in x[0] and 'php' not in x[0] and 'Main_Page' not in x[0]]
# candidate_links = [*set(candidate_links)]

In [ ]:
# len(candidate_links)

In [ ]:
# state_names = ['Alaska', 'Alabama', 'Arkansas', 'American Samoa', 'Arizona', 'California', 'Colorado', 'Connecticut', 'District of Columbia', 'Delaware', 'Florida', 'Georgia', 'Guam', 'Hawaii', 'Iowa', 'Idaho', 'Illinois', 'Indiana', 'Kansas', 'Kentucky', 'Louisiana', 'Massachusetts', 'Maryland', 'Maine', 'Michigan', 'Minnesota', 'Missouri', 'Mississippi', 'Montana', 'North Carolina', 'North Dakota', 'Nebraska', 'New Hampshire', 'New Jersey', 'New Mexico', 'Nevada', 'New York', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Puerto Rico', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Virginia', 'Virgin Islands', 'Vermont', 'Washington', 'Wisconsin', 'West Virginia', 'Wyoming']
# state_regex = '(' + ')|('.join(state_names) + ')'
# party_names = ['Democratic Party','Republican Party','Libertarian Party','Independent','Nonpartisan','Unaffiliated','Alaskan Independence Party','American Independent Party','Green Party','No party preference','Peace and Freedom Party of California','American Constitution Party','Independent Party','No Party Affiliation','Constitution Party','Working Class Party','Legal Marijuana Now Party','Working Families Party','Independence Party','Progressive Party','United Utah Party','Vermont Progressive Party','American Solidarity Party','Congress Sucks Party','Logic Party']
# party_regex = '(' + ')|('.join(party_names) + ')'

In [ ]:
# ballotpedia_scrape = pd.DataFrame(columns = ['name', 'party', 'state', 'district', 'last_election', 'twitter', 'ballotpedia'])

# for url in candidate_links[0:101]:
#     page = requests.get(url)
#     soup = BeautifulSoup(page.content, 'html.parser')
#     body = soup.find('div', class_ = 'mw-parser-output')
#     link_table = soup.find('div', class_ = 'infobox person')
#     name = re.search(r'[A-Za-z\.\' ]+(?=\n\n)', link_table.get_text())[0]
#     try:
#         party = re.search(party_regex, link_table.get_text())[0]
#     except:
#         party = ''
#     try:
#         state = re.search(state_regex, body.get_text())[0]
#     except:
#         state = ''
#     try:
#         district = body.find('a', href = re.compile(str('https://ballotpedia.org/' + state + '\'.*,_2022'))).get_text()
#         district = re.search(str('(?<=' + state + '\')(s | )([0-9]+)'), district)[2]
#     except:
#         district = ''
#     try:
#         last_election = re.search(r'[A-Za-z]+ [0-9]+, [0-9]+(?=\n\n)', link_table.get_text())[0]
#     except:
#         last_election = ''
#     twitter = re.findall(r'(?<=")https://www.twitter.com/\w+(?=")', str(link_table))
#     ballotpedia = url
#     ballotpedia_scrape.loc[len(ballotpedia_scrape)] = [name, party, state, district, last_election, twitter, ballotpedia]
#     # time.sleep(1)
#     if candidate_links.index(url) % 100 == 0:
#         print(f'Progress - {candidate_links.index(url)} of {len(candidate_links)}')